In [1]:
import glob 
import os 
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np

import pandas as pd 
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

from PIL import Image

import p3_model2 as model 

#import p3_hw as hw
import sys



from itertools import cycle
import torch.optim as optim
import p3_test2 as test
import time 

In [2]:
cwd = os.getcwd()
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
target_domain='real' # change here 
#domain_name=['sketch','quickdraw','infograph','real']
domain_name=['infograph']
#domain_name.remove(target_domain)

class_name=[file  for file in os.listdir(domain_name[0]) if file[-3:] !='csv' ]

csv_name_train={name: pd.read_csv(cwd+'/'+name+'/'+name+ '_train.csv',index_col=0) for name in domain_name}
csv_name_val={target_domain: pd.read_csv(cwd+'/'+target_domain+'/'+target_domain+ '_train.csv',index_col=0)}



In [4]:
train_cvs=pd.concat(csv_name_train[key] for key in domain_name)
test_csv=csv_name_val[target_domain]

In [5]:
color_transform=transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])

class finalset(Dataset):
    def __init__(self, train_cvs,train, transform=None):
        """ Intialize the MNIST dataset """
        self.images = None
        self.labels = None
        self.csv=train_cvs
        self.filenames = list(train_cvs.index)
        self.train=train
        self.transform = transform
        self.len = len(self.filenames)                      
    def __getitem__(self, index):

        """ Get a sample from the dataset """
        
        image_fn=self.filenames[index]
        image = Image.open(image_fn)
    
        if image.mode != 'RGB':
            image = np.expand_dims(image, axis=2)
            image=np.concatenate((image,image,image),axis=2)

        if self.transform is not None:
            image = self.transform(image)
        
        if self.train is True:
            label=self.csv.loc[image_fn,'label']
            return image,label,image_fn
        if self.train is False:
            return image, -1, image_fn
    
    def __len__(self):
        return self.len  

train_data  =finalset(train_cvs,train=True,transform=color_transform)
trainloader = DataLoader(train_data, batch_size=64,shuffle=True) 


test_data  =finalset(test_csv,train=True,transform=color_transform)
testloader = DataLoader(test_data, batch_size=64,shuffle=True) 

In [6]:

feature_size=2048
cnn_feature_extractor=torchvision.models.resnet50(pretrained=True) #resnet50 fc is for 1000 calsses
modules = list(cnn_feature_extractor.children())[:-1] # delete the last fc layer.
cnn_feature_extractor = nn.Sequential(*modules).to(device)

# set requires_grad to false
for param in cnn_feature_extractor.parameters():
    param.requires_grad = False
#encoder=model.SVHN_Extractor().cuda()
encoder=cnn_feature_extractor.cuda()
classifier=model.SVHN_Class_classifier().cuda()
discriminator=model.SVHN_Domain_classifier().cuda()


In [7]:
def save_checkpoint(encoder,classifier,discriminator,save_name):
    print('Save models  ...')

    save_folder = 'models'
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    state_enc={"state_dict": encoder.state_dict()}
    torch.save(state_enc, str(save_folder) + '/' + str(save_name) + '_enc.pth')
    
    state_cls={"state_dict": classifier.state_dict()}
    torch.save(state_cls, str(save_folder) + '/' + str(save_name)  +'_cls.pth')
    print(' p3 Model '+ str(save_name)+' is saved !!!')

In [ ]:

srt_time = time.time()
epochs=40
max_target_acc=0
for epoch in range(epochs):
    print("DANN training.......")
    print('Epoch : {}'.format(epoch))

    encoder = encoder.train()
    classifier = classifier.train()
    discriminator = discriminator.train()

    classifier_criterion = nn.NLLLoss().cuda()#nn.CrossEntropyLoss().cuda()
    discriminator_criterion = nn.NLLLoss().cuda()#nn.CrossEntropyLoss().cuda()

    start_steps = epoch * len(trainloader)
    total_steps = epochs * len(testloader)
    lens_use=min(len(trainloader),len(testloader))
    #print(time.time()-srt_time)
    
    target_correct=0
    for batch_idx, (source_data, target_data) in enumerate(zip(trainloader, testloader)):
#         if batch_idx > 50:
#             break 
        str_time=time.time()
        source_image, source_label,_name = source_data
        target_image, target_label ,_name= target_data

        p = float(batch_idx + start_steps) / total_steps
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

            #source_image = torch.cat((source_image, source_image, source_image), 1)

        source_image, source_label = source_image.cuda(), source_label.cuda()
        target_image, target_label = target_image.cuda(), target_label.cuda()
        combined_image = torch.cat((source_image, target_image), 0)

        optimizer = optim.SGD(
            list(encoder.parameters()) +
            list(classifier.parameters()) +
            list(discriminator.parameters()),
            lr=0.01,
            momentum=0.9)

        #optimizer =p3_utils.optimizer_scheduler(optimizer=optimizer, p=p)
        optimizer.zero_grad()

        combined_feature = encoder(combined_image)
        combined_feature=combined_feature.view(-1, feature_size)
        #print(time.time()-str_time)
        source_feature = encoder(source_image)
        source_feature=source_feature.view(-1, feature_size)
        #print(time.time()-str_time)





            # 1.Classification loss
        class_pred = classifier(source_feature)
        pred2 = class_pred.data.max(1, keepdim=True)[1]
        
        target_correct += pred2.eq(source_label.data.view_as(pred2)).cpu().sum()
        

        class_loss = classifier_criterion(class_pred, source_label)
        
            # 2. Domain loss
        domain_pred = discriminator(combined_feature, alpha)
        #print(time.time()-str_time)
        domain_source_labels = torch.zeros(source_label.shape[0]).type(torch.LongTensor)
        domain_target_labels = torch.ones(target_label.shape[0]).type(torch.LongTensor)
        domain_combined_label = torch.cat((domain_source_labels, domain_target_labels), 0).cuda()
        domain_loss = discriminator_criterion(domain_pred, domain_combined_label)
    
        total_loss = class_loss + domain_loss
        #print(time.time()-str_time)
        total_loss.backward()
        optimizer.step()
        #print(time.time()-srt_time)

        if (batch_idx + 1) % 50 == 0:
            print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}\tClass Loss: {:.6f}\tDomain Loss: {:.6f}'.format(
                batch_idx * len(target_image), lens_use*64, 100. * batch_idx / lens_use, total_loss.item(), class_loss.item(), domain_loss.item()))
    print("\nTarget Accuracy: {}/{} \n".format(target_correct,lens_use * 64))
    if (epoch) % 1 == 0:
        source_acc,target_acc=test.tester(encoder, classifier, discriminator, trainloader, testloader)
    
    save_name='wang'
    if(target_acc>max_target_acc):
        save_checkpoint(encoder, classifier, discriminator,  save_name)
        max_target_acc=target_acc
    #visualize(encoder, 'source', save_name)


DANN training.......
Epoch : 0
[3136/37120 (8%)]	Loss: 6.129664	Class Loss: 5.753774	Domain Loss: 0.375890
[6336/37120 (17%)]	Loss: 5.778780	Class Loss: 5.537865	Domain Loss: 0.240915
[9536/37120 (26%)]	Loss: 5.895702	Class Loss: 5.657181	Domain Loss: 0.238521
[12736/37120 (34%)]	Loss: 5.640420	Class Loss: 5.427755	Domain Loss: 0.212665
[15936/37120 (43%)]	Loss: 5.824214	Class Loss: 5.539944	Domain Loss: 0.284270
[19136/37120 (52%)]	Loss: 5.547801	Class Loss: 5.336491	Domain Loss: 0.211310
[22336/37120 (60%)]	Loss: 5.636568	Class Loss: 5.439458	Domain Loss: 0.197110
[25536/37120 (69%)]	Loss: 5.557771	Class Loss: 5.288949	Domain Loss: 0.268823
[28736/37120 (77%)]	Loss: 5.526441	Class Loss: 5.339215	Domain Loss: 0.187226
[31936/37120 (86%)]	Loss: 5.467163	Class Loss: 5.262992	Domain Loss: 0.204171
[35136/37120 (95%)]	Loss: 5.523991	Class Loss: 5.336222	Domain Loss: 0.187769

Target Accuracy: 1504/37120 


Source Accuracy: 490/6464 (7.5804%)


Target Accuracy: 191/6464 (2.9548%)

Domain A